In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

In [3]:
1 / 30

0.03333333333333333

In [10]:
# fmt: off
def moab_model(state, action, dt=0.0333, gravity=9.81, ball_radius=0.02, ball_shell=0.0002):
    r = ball_radius
    h = ball_radius - ball_shell  # hollow radius

    # Ball intertia for a hollow sphere is:
    # I = (2 / 5) * m * ((r**5 - h**5) / (r**3 - h**3))
    # Equations for acceleration on a plate at rest
    # a = (theta * m * g) / (m + (I / r**2))
    # Combine the two to get the acceleration divided by theta
    acc_div_theta = gravity / (1 + (2 / 5) * ((r**5 - h**5) / (r**3 - h**3)) / (r**2))

    A = np.array([[1, 0, dt, 0], [0, 1, 0, dt], [0, 0, 1, 0], [0, 0, 0, 1]])
    B = np.array([
        [(1 / 2) * dt**2 * acc_div_theta, 0],
        [0, (1 / 2) * dt**2 * acc_div_theta],
        [dt * acc_div_theta, 0],
        [0, dt * acc_div_theta]
    ])

    next_state = A @ state + B @ action  # x_t+1 = Ax_t + Bu_t
    return next_state

In [35]:
# fmt: off
def diff_moab_model(state, t, action, gravity, ball_radius, ball_shell):
    h = ball_radius - ball_shell  # hollow radius
    r = ball_radius

    # Ball intertia for a hollow sphere is:
    # I = (2 / 5) * m * ((r**5 - h**5) / (r**3 - h**3))
    # Equations for acceleration on a plate at rest
    # a = (theta * m * g) / (m + (I / r**2))
    # Simplify:
    x_dot_dot, y_dot_dot = np.asarray(action) * (gravity / (1 + (2 / 5) * ((r**5 - h**5) / (r**3 - h**3)) / (r**2)))
    _, _, x_dot, y_dot = state
    return np.array([x_dot, y_dot, x_dot_dot, y_dot_dot])

def moab_model_ode(state, action, dt=0.0333, gravity=9.81, ball_radius=0.02, ball_shell=0.0002):
    # diff_model = lambda s, a: diff_moab_model(s, a, gravity, ball_radius, ball_shell, diff_model)
    t = np.linspace(0.0, dt, 2)
    ode_result = odeint(diff_moab_model, state, t, args=(action, gravity, ball_radius, ball_shell))
    # next_state = odeint(diff__model_ode, state, t, args=(Vm, dt))

    return np.array(ode_result[1, :])

In [37]:
%timeit moab_model([1, 2, 2, 1], [0.2, 0.2])

8.29 µs ± 48.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [38]:
%timeit moab_model_ode([1, 2, 2, 1], [0.2, 0.2])

38.1 µs ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [68]:
def mm1(state=np.random.randn(4), actions=np.random.randn(2, 50), dt=0.033):
    n = actions.shape[1]
    a = b = state
    for i in range(n):
        a = moab_model(a, actions[:, i], dt=dt)
    return a


def mm2(state=np.random.randn(4), actions=np.random.randn(2, 50), dt=0.033):
    n = actions.shape[1]
    dt = 0.033
    a = b = state
    for i in range(n):
        b = moab_model_ode(b, actions[:, i], dt=dt)
    return b


state = np.random.randn(4)
actions = np.random.randn(2, 10000)
mm1(state, actions), mm2(state, actions)

(array([  683.64319516, -6777.94140649,    18.46499925,   -31.1310887 ]),
 array([  683.64319517, -6777.9414065 ,    18.46499925,   -31.1310887 ]))

In [57]:
%timeit mm1()

331 µs ± 1.23 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [58]:
%timeit mm2()

1.72 ms ± 9.14 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
